In [8]:
import requests
import re
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import wordnet

## Synonyms with Thesaurus API

In [9]:
keys = ["Production", "Retail", "Processing", "Distribution"]
#no_keys = [ "consumer discretionary", "consumer staples", "communication services"]

In [11]:
master_list = {}
i = 0
for word in keys:
    x = requests.get(f"https://www.dictionaryapi.com/api/v3/references/thesaurus/json/{word}?key=")
    data = x.json()
    master_list[keys[i]] = data
    i += 1

master_list.keys()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Problems with some words

None of these words work with the API (DO NOT WORK)

    -> they do not exist within the Thesaurus, therefore the JSON returns a list with possible words
Definition of Consumer Discretionary:
- goods that are non-essential but desirable if their income is sufficient to purchase them

Definition of Consumer Staples:
- goods that are essential

Definition of Communication Services:
- elecommunications Services, Cable Services, Video Services, or Information Services

-> Solution for now: remove those words from here

In [6]:
# consumer discretionary, consumer staples, communication services
master_list["Production"][0]#["def"]#.keys()

{'meta': {'id': 'production',
  'uuid': 'd8e5a3d3-0ede-461f-b2c7-a84c6a287e6d',
  'sort': '167730000',
  'src': 'collegiate',
  'section': 'alpha',
  'stems': ['production', 'productional', 'productions'],
  'offensive': False},
 'hwi': {'hw': 'pro*duc*tion',
  'prs': [{'mw': 'prə-ˈdək-shən',
    'sound': {'audio': 'produc07', 'ref': 'c', 'stat': '1'}},
   {'mw': 'prō-'}]},
 'fl': 'noun',
 'def': [{'sseq': [[['sense',
      {'sn': '1 a',
       'dt': [['text',
         '{bc}something {a_link|produced} {bc}{sx|product||}']]}],
     ['pseq',
      [['sense',
        {'sn': 'b (1)', 'dt': [['text', '{bc}a literary or artistic work']]}],
       ['sense',
        {'sn': '(2)',
         'dt': [['text',
           '{bc}a work presented to the public (as on the stage or screen or over the air)']]}]]],
     ['sense',
      {'sn': 'c',
       'dt': [['text',
         '{bc}something exaggerated out of proportion to its importance']]}]],
    [['sense',
      {'sn': '2 a',
       'dt': [['text', '{

In [7]:
# check that each word within master_list contains the "syns"
keys_within_keys = [master_list[key][0].keys() for key in master_list]
keys_within_keys

[dict_keys(['meta', 'hwi', 'fl', 'def', 'uros', 'date', 'shortdef']),
 dict_keys(['meta', 'hom', 'hwi', 'fl', 'ins', 'def', 'uros', 'date', 'shortdef']),
 dict_keys(['meta', 'hom', 'hwi', 'fl', 'ins', 'def', 'date', 'shortdef']),
 dict_keys(['meta', 'hwi', 'fl', 'def', 'uros', 'date', 'shortdef'])]

## Synonyms with NLP

In [ ]:
def get_synonyms(phr):
    phr_syn_lst = [phr]
    phr_syn_lst = [i.name().lower() for syn in wordnet.synsets(phr) for i in syn.lemmas() ]

    return (list(set(phr_syn_lst)))    

In [ ]:
get_synonyms("agree")

## Algorithm

In [ ]:
df = pd.read_csv("scopus_2k.csv")#, on_bad_lines='skip')
df.columns

In [ ]:
df['Abstract'] = df['Abstract'].str.lower()
df['Title'] = df['Title'].str.lower()
df['Author Keywords'] = df['Author Keywords'].str.lower()
df['Index Keywords'] = df['Index Keywords'].str.lower()

In [ ]:
df.info()

In [ ]:
df = df.fillna("blank")

In [ ]:
df.head(2)

In [ ]:
for col in ['Title','Abstract', 'Author Keywords', 'Index Keywords']:
    df[col] = df[col].map(lambda x: x.replace(';',''))
    df[col] = df[col].map(lambda x: x.replace(',',''))
    df[col] = df[col].map(lambda x: x.replace(')',''))
    df[col] = df[col].map(lambda x: x.replace('(',''))

In [ ]:
df['content_sum'] = df['Abstract'] + df['Title'] + df['Author Keywords'] + df['Index Keywords']
df['content_sum'].values[0].split()

In [ ]:
df_copy = df.copy()

In [ ]:
key_words = ["energy", "materials", "industrials", "consumer discretionary", "consumer staples", "health care", "financials", "information technology", "communication services", "utilities", "real estate"]

In [ ]:
for key in key_words:
    df[key] = 0
    for phr in get_synonyms(key):
        for i in range(len(df)): 
            if phr in (df['content_sum'].values[i].split()):
                df[key][i] += 1
    #df.loc[key, i] = count

In [ ]:
df